In [179]:
import pandas as pd
from pandas import get_dummies
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
sns.set(color_codes=True)
sns.set(style='white',palette='muted')

from matplotlib.ticker import StrMethodFormatter

from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet, BayesianRidge, SGDRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

In [180]:
df = pd.read_csv('C:\capstone\mongo\output.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\601828809.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('C:\capstone\mongo\output.csv')


## Preprocessing

In [181]:
df.columns
df2=df

In [182]:
df.dtypes

Unnamed: 0                                 int64
result.name.first                         object
result.name.last                          object
result.username                           object
result.password                           object
result.phoneNumber                        object
result.location.street                    object
result.location.city                      object
result.location.state                     object
result.location.country                   object
result.location.zip                       object
result.location.coordinates.latitude     float64
result.location.coordinates.longitude    float64
result.job                                object
result.salary                            float64
result.objectId                           object
id                                         int64
dtype: object

In [183]:
# renaming columns
df2.columns = df2.columns.str.replace('result.', '')
df2.columns = df2.columns.str.replace('.', '_')


In [184]:
# count
df2.shape[0]

1200

In [185]:
# checking for duplicates
duplicate = df2[df2.duplicated(['name_first','name_last'])]
duplicate.head()


,Unnamed: 0,name_first,name_last,username,password,phoneNumber,location_street,location_city,location_state,location_country,location_zip,location_coordinates_latitude,location_coordinates_longitude,job,salary,objectId,id
782,182,Napoleon,Parisian,Napoleon-Parisian,1RPuC6xASl8MTXd,401.300.0279 x9157,2041 McLaughlin Manor,Madison,North Carolina,Democratic Republic of the Congo,14157,22.3259,-94.8501,Corporate Integration Technician,251.0,66916acdc5d023f89ccba720,782


In [186]:
# # dropping duplicates
# df3=df2.drop_duplicates(subset=['name_first','name_last'])
# df3.shape[0]

In [187]:
#checking salary distribution
df3=df2
df3['salary'] = df3['salary'].apply(lambda x: x*100)
df3.boxplot(column =['salary'], grid = True)

<Axes: >

In [188]:
# checking for nulls
print(df3.isnull().sum())

Unnamed: 0                        0
name_first                        0
name_last                         0
username                          0
password                          0
phoneNumber                       0
location_street                   0
location_city                     0
location_state                    0
location_country                  0
location_zip                      0
location_coordinates_latitude     0
location_coordinates_longitude    0
job                               0
salary                            0
objectId                          0
id                                0
dtype: int64


In [189]:
df3.columns

Index(['Unnamed: 0', 'name_first', 'name_last', 'username', 'password',
       'phoneNumber', 'location_street', 'location_city', 'location_state',
       'location_country', 'location_zip', 'location_coordinates_latitude',
       'location_coordinates_longitude', 'job', 'salary', 'objectId', 'id'],
      dtype='object')

In [190]:
features_df=df3[['location_state','job','salary']]
features_df.head(3)

,location_state,job,salary
0,Michigan,Senior Usability Consultant,21700.0
1,Tennessee,National Response Representative,11400.0
2,Minnesota,Legacy Directives Analyst,42100.0


In [191]:
features_df.columns

Index(['location_state', 'job', 'salary'], dtype='object')

## Linear Regression

In [192]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {lr_model.coef_}")
# print(f"Intercept: {lr_model.intercept_}")
print(f"Mean squared error: {mse}")

Mean squared error: 731132038.2603713


C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\2957863656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\2957863656.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


In [193]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = lr_model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

Predicted salary: 48107.75


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## Logistic Regression

In [194]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
lg_model = LogisticRegression()
lg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lg_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {model.coef_}")
# print(f"Intercept: {model.intercept_}")
print(f"Mean squared error: {mse}")

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\3315101942.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\3315101942.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


Mean squared error: 2218344000.0


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [195]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = lg_model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

Predicted salary: 25300.00


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## Ridge

In [196]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
rdg_model = Ridge(alpha=0.1)
rdg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rdg_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {model.coef_}")
# print(f"Intercept: {model.intercept_}")
print(f"Mean squared error: {mse}")

Mean squared error: 731132038.5405833


C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\1935957873.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\1935957873.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


In [197]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = rdg_model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

Predicted salary: 48107.75


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


## Lasso

In [198]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
ls_model = Lasso(alpha=0.1)
ls_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ls_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {model.coef_}")
# print(f"Intercept: {model.intercept_}")
print(f"Mean squared error: {mse}")

Mean squared error: 731132078.1118878


C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\67176356.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\67176356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


In [199]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = ls_model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

Predicted salary: 48107.74


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


## Elastic Net Regression

In [200]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
en_model = ElasticNet(alpha=0.1)
en_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = en_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {model.coef_}")
# print(f"Intercept: {model.intercept_}")
print(f"Mean squared error: {mse}")

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\335408979.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\335408979.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


Mean squared error: 731132192.6507299


In [201]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = en_model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

Predicted salary: 48107.72


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


## Bayesian Ridge Regression

In [202]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

df=features_df

# Encode categorical features (job and location)
le_job = LabelEncoder()
le_location = LabelEncoder()
df['job_encoded'] = le_job.fit_transform(df['job'])
df['location_encoded'] = le_location.fit_transform(df['location_state'])

# Split data into train and test sets
X = df[['job_encoded', 'location_encoded']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = BayesianRidge()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the model coefficients and mean squared error
# print(f"Model coefficients: {model.coef_}")
# print(f"Intercept: {model.intercept_}")
print(f"Mean squared error: {mse}")

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\2247552724.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_encoded'] = le_job.fit_transform(df['job'])
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_3036\2247552724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_encoded'] = le_location.fit_transform(df['location_state'])


Mean squared error: 729360431.9842308


In [203]:
new_job_encoded = le_job.transform(['Central Applications Administrator'])[0]
new_location_encoded = le_location.transform(['Texas'])[0]
predicted_salary = model.predict([[new_job_encoded, new_location_encoded]])[0]
print(f"Predicted salary: {predicted_salary:.2f}")

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Predicted salary: 49385.58


## Pickle File

In [204]:
import pickle

# Specify the file path to save the pipeline
lr_pipeline = 'lr_pipeline.pkl'
lg_pipeline = 'lg_pipeline.pkl'
rdg_pipeline = 'rdg_pipeline.pkl'
en_pipeline = 'en_pipeline.pkl'


# Save the pipeline to disk
with open(lr_pipeline, 'wb') as file:
    pickle.dump(lr_model, file)
with open(lg_pipeline, 'wb') as file:
    pickle.dump(lg_model, file)
with open(rdg_pipeline, 'wb') as file:
    pickle.dump(rdg_model, file)
with open(en_pipeline, 'wb') as file:
    pickle.dump(en_model, file)